In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [2]:
import locale

locale._override_localeconv["thousands_sep"] = ","
locale._override_localeconv["mon_thousands_sep"] = ","

In [3]:
data=pd.read_csv("train_data.csv")
data.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,...,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
0,1,Monthly,Arrear,Closed,PDC_E,1,NaN,48,450000,275000.0,...,1568,21946,61.11,M,49.0,35833.33,RAISEN,MADHYA PRADESH,464993.0,> 48 Months
1,2,Monthly,Advance,Closed,PDC,333,BHOPAL,47,485000,350000.0,...,1062,34802,70.00,M,23.0,666.67,SEHORE,MADHYA PRADESH,466001.0,No Top-up Service
2,3,Quatrly,Arrear,Active,Direct Debit,1,NaN,68,690000,519728.0,...,1060,127335,69.77,M,39.0,45257.00,BHOPAL,MADHYA PRADESH,462030.0,12-18 Months
3,7,Monthly,Advance,Closed,Billed,125,GUNA,48,480000,400000.0,...,1060,25094,80.92,M,24.0,20833.33,ASHOK NAGAR,MADHYA PRADESH,473335.0,> 48 Months
4,8,Monthly,Arrear,Closed,Billed,152,BILASPUR,44,619265,440000.0,...,1046,21853,71.05,M,56.0,27313.67,BILASPUR,CHATTISGARH,495442.0,36-48 Months


In [4]:
data.shape

(128655, 26)

In [5]:
data.describe()

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE
count,128655.000000,128655.000000,128655.000000,1.286550e+05,1.286550e+05,1.286550e+05,128655.000000,1.286550e+05,128655.000000,128655.000000,128655.000000,128596.000000,1.284210e+05,128283.000000
mean,70965.326556,166.289674,42.464778,5.550247e+05,3.483096e+05,3.479306e+05,55072.758506,1.639773e+07,1171.327271,56551.226210,59.126226,40.664803,5.032360e+04,427931.099725
std,41762.779285,115.844048,23.533972,1.083036e+05,1.055453e+05,1.053198e+05,28910.111740,9.539540e+06,257.790208,36399.355276,15.539038,11.712847,1.724607e+06,175704.364031
min,1.000000,1.000000,5.000000,2.000000e+05,5.000000e+04,2.894000e+03,0.000000,4.222710e+05,1019.000000,5879.000000,-1.380000,18.000000,0.000000e+00,110000.000000
25%,34408.500000,50.000000,36.000000,5.000000e+05,2.900000e+05,2.900000e+05,32500.000000,9.438948e+06,1046.000000,24093.000000,49.140000,31.000000,2.500000e+04,304804.000000
50%,70988.000000,152.000000,36.000000,5.507350e+05,3.500000e+05,3.500000e+05,59700.000000,1.513393e+07,1062.000000,39146.000000,60.780000,40.000000,3.583333e+04,458553.000000
75%,106549.500000,274.000000,48.000000,6.110000e+05,4.100000e+05,4.100000e+05,73800.000000,2.533396e+07,1186.000000,87034.000000,70.590000,49.000000,5.000000e+04,521131.000000
max,143395.000000,424.000000,501.000000,2.250000e+06,1.308351e+06,1.308351e+06,460000.000000,3.706667e+07,3473.000000,145518.000000,100.000000,90.000000,6.174775e+08,855456.000000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128655 entries, 0 to 128654
Data columns (total 26 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ID               128655 non-null  int64  
 1   Frequency        128655 non-null  object 
 2   InstlmentMode    128655 non-null  object 
 3   LoanStatus       128655 non-null  object 
 4   PaymentMode      128655 non-null  object 
 5   BranchID         128655 non-null  int64  
 6   Area             117002 non-null  object 
 7   Tenure           128655 non-null  int64  
 8   AssetCost        128655 non-null  int64  
 9   AmountFinance    128655 non-null  float64
 10  DisbursalAmount  128655 non-null  float64
 11  EMI              128655 non-null  float64
 12  DisbursalDate    128655 non-null  object 
 13  MaturityDAte     128654 non-null  object 
 14  AuthDate         128655 non-null  object 
 15  AssetID          128655 non-null  int64  
 16  ManufacturerID   128655 non-null  int6

In [7]:
data.tail()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,...,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
128650,143390,Half Yearly,Arrear,Closed,Direct Debit,424,PANIPAT,24,470000,265601.0,...,1568,48879,40.17,M,25.0,65333.33,SONIPAT,HARYANA,131403.0,24-30 Months
128651,143391,Half Yearly,Arrear,Closed,Direct Debit,424,PANIPAT,24,460000,275630.0,...,1568,48879,59.92,M,25.0,83333.33,SONIPAT,HARYANA,131403.0,No Top-up Service
128652,143393,Monthly,Arrear,Active,Direct Debit,424,PANIPAT,23,545000,300733.0,...,1568,44118,52.38,M,36.0,248500.00,SONIPAT,HARYANA,131024.0,No Top-up Service
128653,143394,Half Yearly,Arrear,Active,Direct Debit,424,PANIPAT,35,350000,250962.0,...,1568,48879,50.37,M,37.0,84500.00,SONIPAT,HARYANA,131103.0,No Top-up Service
128654,143395,Half Yearly,Arrear,Active,Direct Debit,424,PANIPAT,24,370000,200428.0,...,1568,48879,54.17,M,33.0,178166.67,SONIPAT,HARYANA,131402.0,No Top-up Service


In [8]:
data['DisbursalDate'] = pd.to_datetime(data['DisbursalDate'])
data['MaturityDAte'] = pd.to_datetime(data['MaturityDAte'])
data['DisbursalYear'] = data['DisbursalDate'].dt.year
data['MaturityYear'] = data['MaturityDAte'].dt.year

In [9]:
data.isnull().sum()

ID                     0
Frequency              0
InstlmentMode          0
LoanStatus             0
PaymentMode            0
BranchID               0
Area               11653
Tenure                 0
AssetCost              0
AmountFinance          0
DisbursalAmount        0
EMI                    0
DisbursalDate          0
MaturityDAte           1
AuthDate               0
AssetID                0
ManufacturerID         0
SupplierID             0
LTV                    0
SEX                   59
AGE                   59
MonthlyIncome        234
City               11256
State                  0
ZiPCODE              372
Top-up Month           0
DisbursalYear          0
MaturityYear           1
dtype: int64

In [10]:
data.isnull().sum().sum()

23635

In [11]:
data['AGE'] = data['AGE'].fillna(data['AGE'].mean())
data['MonthlyIncome'] = data['MonthlyIncome'].fillna(data['MonthlyIncome'].mean())
data['MaturityYear'].fillna(data['MaturityYear'].mode()[0], inplace=True)
data['SEX'].fillna(method='ffill', inplace=True)

C:\Users\Rithu Varshini V K\AppData\Local\Temp\ipykernel_44576\3771412153.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['SEX'].fillna(method='ffill', inplace=True)


In [12]:
data.isnull().sum()

ID                     0
Frequency              0
InstlmentMode          0
LoanStatus             0
PaymentMode            0
BranchID               0
Area               11653
Tenure                 0
AssetCost              0
AmountFinance          0
DisbursalAmount        0
EMI                    0
DisbursalDate          0
MaturityDAte           1
AuthDate               0
AssetID                0
ManufacturerID         0
SupplierID             0
LTV                    0
SEX                    0
AGE                    0
MonthlyIncome          0
City               11256
State                  0
ZiPCODE              372
Top-up Month           0
DisbursalYear          0
MaturityYear           0
dtype: int64

In [13]:
def convert_target(topup):
    if topup == "No Top-up Service" : 
        return 0
    elif topup == "12-18 Months" :
        return 1
    elif topup == "18-24 Months" :
        return 2
    elif topup == "24-30 Months" :
        return 3
    elif topup == "30-36 Months" :
        return 4
    elif topup == "36-48 Months" :
        return 5
    elif topup == " > 48 Months" :
        return 6
        
def reconvert_target(topup):
    if topup == 0  : 
        return "No Top-up Service"
    elif topup == 1 :
        return "12-18 Months"
    elif topup == 2 :
        return "18-24 Months"
    elif topup == 3 :
        return "24-30 Months"
    elif topup == 4 :
        return "30-36 Months"
    elif topup == 5 :
        return "36-48 Months"
    elif topup == 6 :
        return " > 48 Months"

def get_1st_value(val):
  if len(val)>0 :
    return val.iloc[0]
  else :
    return ''

def convert_toZero(val):  
  if val == '' :
    return 0
  else :
    if type(val) == str :
      return locale.atof(val)
    else :
      return val

def convert_ownership(ownership):
  if ownership == 'Individual' :
    return 1
  elif ownership == 'Guarantor' :
    return 2
  elif ownership == 'Joint' :
    return 3
  else :
    return 0

def convert_accttype(accttype):
  if accttype == 'Tractor Loan' :
    return 1
  elif accttype == 'Business Loan Priority Sector  Agriculture' :
    return 2
  elif accttype == 'Business Loan General' :
    return 3
  else :
    return 0

def convert_sex(sex) :
    if sex == 'M' :
        return 1
    elif sex == 'F' :
        return 0

def binning_amountfinanced(amountfinanced):
    return math.ceil((amountfinanced/100000))

def binning_assetcost(assetcost):
    return math.ceil((assetcost/100000))

def binning_tenure(tenure):
    if tenure < 12 : 
        return 1
    elif tenure >12 and tenure <= 18 :
        return 1.5
    elif tenure >18 and tenure <= 24 :
        return 2
    elif tenure >24 and tenure <= 30 :
        return 2.5
    elif tenure >30 and tenure <= 36 :
        return 3
    elif tenure >36 and tenure <= 48 :
        return 4
    elif tenure >48 and tenure <= 60 :
        return 5
    elif tenure >60 and tenure <= 120 :
        return 10
    elif tenure >120  :
        return 20

def convert_loanstatus(loanstatus):
    if loanstatus == "Closed" : 
        return 0
    elif loanstatus == "Active" :
        return 1

def convert_instalmentmode(instlmentmode):
    if instlmentmode == "Arrear" : 
        return 0
    elif instlmentmode == "Advance" :
        return 1

def convert_frequency(frequency):
    if frequency == "BI-Monthly" : 
        return 0
    elif frequency == "Monthly" :
        return 1
    elif frequency == "Quatrly" :
        return 2
    elif frequency == "Half Yearly" :
        return 3



In [14]:
data.drop(columns=['PaymentMode', 'BranchID', 'Area','DisbursalAmount', 'AuthDate', 'AssetID','ManufacturerID',
                   'SupplierID', 'City', 'State', 'ZiPCODE','AuthDate','MaturityDAte','Area'],inplace=True)

In [15]:
data['Frequency'] = data['Frequency'].apply(convert_frequency)
data['InstlmentMode'] = data['InstlmentMode'].apply(convert_instalmentmode)
data['LoanStatus'] = data['LoanStatus'].apply(convert_loanstatus)
data['Tenure'] = data['Tenure'].apply(binning_tenure)
data['AssetCost'] = data['AssetCost'].apply(binning_assetcost)
data['AmountFinance'] = data['AmountFinance'].apply(binning_amountfinanced)
data['SEX'] = data['SEX'].apply(convert_sex)

In [16]:
data_B=pd.read_csv("train_bureau.csv")
data_B.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,...,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,1,False,PRIMARY,Overdraft,NAB,2018-04-30 00:00:00,Individual,Delinquent,2015-10-05 00:00:00,NaN,...,NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN
1,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31 00:00:00,Individual,Active,2018-03-19 00:00:00,NaN,...,F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",36.0
2,1,True,PRIMARY,Tractor Loan,NBF,2020-01-31 00:00:00,Individual,Active,2019-08-30 00:00:00,NaN,...,NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",NaN
3,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30 00:00:00,Individual,Closed,2013-09-27 00:00:00,2017-09-21 00:00:00,...,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN
4,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29 00:00:00,Individual,Closed,2012-02-10 00:00:00,2016-02-01 00:00:00,...,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN


In [17]:
data_B.shape

(560844, 25)

In [18]:
data_B.describe()

,ID,WRITE-OFF-AMT,TENURE
count,560844.000000,5.417210e+05,192737.000000
mean,67401.698360,2.956319e+02,28.801029
std,41016.177296,1.950165e+04,32.170975
min,1.000000,-3.000000e+01,0.000000
25%,32434.000000,0.000000e+00,12.000000
50%,61399.000000,0.000000e+00,23.000000
75%,101283.250000,0.000000e+00,36.000000
max,143395.000000,8.797356e+06,856.000000


In [19]:
data_B.info

<bound method DataFrame.info of             ID  SELF-INDICATOR MATCH-TYPE             ACCT-TYPE  \
0            1           False    PRIMARY             Overdraft   
1            1           False    PRIMARY  Auto Loan (Personal)   
2            1            True    PRIMARY          Tractor Loan   
3            1            True    PRIMARY  Auto Loan (Personal)   
4            1            True    PRIMARY          Tractor Loan   
...        ...             ...        ...                   ...   
560839  143393           False    PRIMARY          Tractor Loan   
560840  143393           False    PRIMARY          Tractor Loan   
560841  143393           False    PRIMARY  Auto Loan (Personal)   
560842  143394            True    PRIMARY          Tractor Loan   
560843  143395            True    PRIMARY          Tractor Loan   

       CONTRIBUTOR-TYPE        DATE-REPORTED OWNERSHIP-IND ACCOUNT-STATUS  \
0                   NAB  2018-04-30 00:00:00    Individual     Delinquent   
1        

In [20]:
data_B.isnull().sum()

ID                                0
SELF-INDICATOR                    0
MATCH-TYPE                        0
ACCT-TYPE                         0
CONTRIBUTOR-TYPE                  0
DATE-REPORTED                  3683
OWNERSHIP-IND                     0
ACCOUNT-STATUS                    0
DISBURSED-DT                  32150
CLOSE-DT                     251758
LAST-PAYMENT-DATE            319283
CREDIT-LIMIT/SANC AMT        545685
DISBURSED-AMT/HIGH CREDIT         0
INSTALLMENT-AMT              420509
CURRENT-BAL                     233
INSTALLMENT-FREQUENCY        425135
OVERDUE-AMT                  118891
WRITE-OFF-AMT                 19123
ASSET_CLASS                  300376
REPORTED DATE - HIST          19123
DPD - HIST                    19647
CUR BAL - HIST                19123
AMT OVERDUE - HIST            19123
AMT PAID - HIST               20294
TENURE                       368107
dtype: int64

In [21]:
data['ACC-TYP_Present'] = ""
data['OWNERSHIP-IND_Present'] = ""
data['CURRENT-BAL_Present'] = "" 
data['OVERDUE-AMT_Present'] =""
data['total_count_Own']= ""

data['count_distinct_accttype_Own'] =""
data['count_active_Own'] =""
data['count_closed_Own']=""
data['count_delinquent_Own']=""
data['count_others_Own']=""

data['OTHER_total_count']=""
data['OTHER_count_distinct_accttype'] =""
data['OTHER_count_active'] =""
data['OTHER_count_closed']=""
data['OTHER_count_delinquent']=""
data['OTHER_count_others']=""

acct_not_others = ['Active','Closed', 'Delinquent']

In [22]:
for index, row in data.iterrows() :
  temp_ID = row['ID']
  temp_DisbDate = row['DisbursalDate']
  
  data_B_temp = data_B.loc[data_B['ID'] == temp_ID]
  data_B_temp_Present = data_B_temp.loc[(data_B_temp['SELF-INDICATOR']== True) & (data_B_temp['DISBURSED-DT']== temp_DisbDate)]
  data.at[index,'ACC-TYP_Present'] = get_1st_value(data_B_temp_Present['ACCT-TYPE'])
  data.at[index,'OWNERSHIP-IND_Present'] = get_1st_value(data_B_temp_Present['OWNERSHIP-IND'])
  data.at[index,'CURRENT-BAL_Present'] = get_1st_value(data_B_temp_Present['CURRENT-BAL'])
  data.at[index,'OVERDUE-AMT_Present'] = get_1st_value(data_B_temp_Present['OVERDUE-AMT'])

  data_B_temp_Own = data_B_temp.loc[(data_B_temp['SELF-INDICATOR']== True) & (data_B_temp['DISBURSED-DT']!= temp_DisbDate)]
  data.at[index,'total_count_Own'] = len(data_B_temp_Own.index)
  data.at[index,'count_distinct_accttype_Own'] = data_B_temp_Own['ACCT-TYPE'].nunique()
  data.at[index,'count_active_Own'] =len(data_B_temp_Own[data_B_temp_Own['ACCOUNT-STATUS'] == 'Active'])
  data.at[index,'count_closed_Own']=len(data_B_temp_Own[data_B_temp_Own['ACCOUNT-STATUS'] == 'Closed'])
  data.at[index,'count_delinquent_Own']=len(data_B_temp_Own[data_B_temp_Own['ACCOUNT-STATUS'] == 'Delinquent'])
  data.at[index,'count_others_Own']= len(data_B_temp_Own[~data_B_temp_Own['ACCOUNT-STATUS'].isin(acct_not_others)])

  data_B_temp_OTHER = data_B_temp.loc[data_B_temp['SELF-INDICATOR']== False]
  data.at[index,'OTHER_total_count'] = len(data_B_temp_OTHER.index)
  data.at[index,'OTHER_count_distinct_accttype'] = data_B_temp_OTHER['ACCT-TYPE'].nunique()
  data.at[index,'OTHER_count_active'] =len(data_B_temp_OTHER[data_B_temp_OTHER['ACCOUNT-STATUS'] == 'Active'])
  data.at[index,'OTHER_count_closed']=len(data_B_temp_OTHER[data_B_temp_OTHER['ACCOUNT-STATUS'] == 'Closed'])
  data.at[index,'OTHER_count_delinquent']=len(data_B_temp_OTHER[data_B_temp_OTHER['ACCOUNT-STATUS'] == 'Delinquent'])
  data.at[index,'OTHER_count_others']= len(data_B_temp_OTHER[~data_B_temp_OTHER['ACCOUNT-STATUS'].isin(acct_not_others)])

In [23]:
data['ACC-TYP_Present'] = data['ACC-TYP_Present'].apply(convert_accttype)
data['OWNERSHIP-IND_Present'] = data['OWNERSHIP-IND_Present'].apply(convert_ownership)

data['CURRENT-BAL_Present'] = data['CURRENT-BAL_Present'].apply(convert_toZero)
data['OVERDUE-AMT_Present'] = data['OVERDUE-AMT_Present'].apply(convert_toZero)

data['Top-up Month'] = data['Top-up Month'].apply(convert_target)

In [24]:
data.drop(columns=['DisbursalDate'],inplace=True)
data.fillna(0,inplace=True)
data.drop(columns=['ID'],inplace=True)

In [25]:
data.to_csv('data_processed.csv',index=False)

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128655 entries, 0 to 128654
Data columns (total 30 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Frequency                      128655 non-null  int64  
 1   InstlmentMode                  128655 non-null  int64  
 2   LoanStatus                     128655 non-null  int64  
 3   Tenure                         128655 non-null  float64
 4   AssetCost                      128655 non-null  int64  
 5   AmountFinance                  128655 non-null  int64  
 6   EMI                            128655 non-null  float64
 7   LTV                            128655 non-null  float64
 8   SEX                            128655 non-null  int64  
 9   AGE                            128655 non-null  float64
 10  MonthlyIncome                  128655 non-null  float64
 11  Top-up Month                   128655 non-null  int64  
 12  DisbursalYear                 

In [27]:
data.shape

(128655, 30)

In [28]:
data.isnull().sum()

Frequency                        0
InstlmentMode                    0
LoanStatus                       0
Tenure                           0
AssetCost                        0
AmountFinance                    0
EMI                              0
LTV                              0
SEX                              0
AGE                              0
MonthlyIncome                    0
Top-up Month                     0
DisbursalYear                    0
MaturityYear                     0
ACC-TYP_Present                  0
OWNERSHIP-IND_Present            0
CURRENT-BAL_Present              0
OVERDUE-AMT_Present              0
total_count_Own                  0
count_distinct_accttype_Own      0
count_active_Own                 0
count_closed_Own                 0
count_delinquent_Own             0
count_others_Own                 0
OTHER_total_count                0
OTHER_count_distinct_accttype    0
OTHER_count_active               0
OTHER_count_closed               0
OTHER_count_delinque

In [29]:
print(data['Top-up Month'])

0         6
1         0
2         1
3         6
4         5
         ..
128650    3
128651    0
128652    0
128653    0
128654    0
Name: Top-up Month, Length: 128655, dtype: int64


In [39]:
columns_to_convert = [
    'total_count_Own', 'count_distinct_accttype_Own', 'count_active_Own', 
    'count_closed_Own', 'count_delinquent_Own', 'count_others_Own',
    'OTHER_total_count', 'OTHER_count_distinct_accttype', 'OTHER_count_active', 
    'OTHER_count_closed', 'OTHER_count_delinquent', 'OTHER_count_others'
]

for col in columns_to_convert:
    data[col] = pd.to_numeric(data[col], errors='coerce').astype('Int64')

In [40]:
train_data, test2_data = train_test_split(data, test_size=0.15, random_state=42)
Org_train,test1_data= train_test_split(train_data, test_size=(15/85), random_state=42)

test1_data.to_csv('test1_data.csv', index=False)
test2_data.to_csv('test2_data.csv', index=False)
Org_train.to_csv('Org_train.csv',index=False)

print("Final train data size: ", Org_train.shape)
print("Test1 data size: ", test1_data.shape)
print("Test2 data size: ", test2_data.shape)


Final train data size:  (90057, 30)
Test1 data size:  (19299, 30)
Test2 data size:  (19299, 30)


In [41]:
X = Org_train.drop('Top-up Month', axis=1)
y = Org_train['Top-up Month']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [44]:
model1 = LogisticRegression(max_iter=1000)
model2 = RandomForestClassifier()
model3 = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model4=SVC()
model5=GaussianNB()
model6=KNeighborsClassifier()

model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)
f1_LR = f1_score(y_test, y_pred1, average='macro')

model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)
f1_RF = f1_score(y_test, y_pred2, average='macro')

model3.fit(X_train, y_train)
y_pred3 = model3.predict(X_test)
f1_XG = f1_score(y_test, y_pred3, average='macro')

model4.fit(X_train, y_train)
y_pred4 = model4.predict(X_test)
f1_SVM = f1_score(y_test, y_pred4, average='macro')

model5.fit(X_train, y_train)
y_pred5 = model5.predict(X_test)
f1_NB = f1_score(y_test, y_pred5, average='macro')

model6.fit(X_train, y_train)
y_pred6 = model6.predict(X_test)
f1_KN = f1_score(y_test, y_pred6, average='macro')

print(f"F1 Score for Logistic Regression: {f1_LR}")
print(f"F1 Score for Random Forest: {f1_RF}")
print(f"F1 Score for XGBoost: {f1_XG}")
print(f"F1 Score for SVM: {f1_SVM}")
print(f"F1 Score for Gaussian: {f1_NB}")
print(f"F1 Score for KNeighbors: {f1_KN}")

C:\Users\Rithu Varshini V K\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Rithu Varshini V K\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


F1 Score for Logistic Regression: 0.12921611899838392
F1 Score for Random Forest: 0.3631238204804722
F1 Score for XGBoost: 0.3836277103444578
F1 Score for SVM: 0.12921992618387307
F1 Score for Gaussian: 0.026832693041865443
F1 Score for KNeighbors: 0.13240189979529288
